# Week 6 worksheet 1: Introduction to numerical integration

This notebook is modified from one created by Charlotte Desvages.

This week, we investigate numerical methods to estimate integrals.

The best way to learn programming is to write code. Don't hesitate to edit the code in the example cells, or add your own code, to test your understanding. You will find practice exercises throughout the notebook, denoted by 🚩 ***Exercise $x$:***.

#### Displaying solutions

Solutions will be released one week after the worksheets are released, as a new `.txt` file in the same GitHub repository. After pulling the file to your workspace, run the following cell to create clickable buttons under each exercise, which will allow you to reveal the solutions.

In [ ]:
%run scripts/create_widgets.py W06-W1

## Numerical integration

🚩 *Recommended reading:* Section 3.3 in **ASC**

Numerical integration is the process of computing an approximation of a definite integral, using a particular *scheme*. There are many different ways we could go about this, but in general, we want to approximate an integral using a **weighted sum** which is easy to compute:

$$
\int_a^b f(x) \ dx \approx \sum_{k=0}^{N-1} w_k f(x_k),
$$

where
- $x_k \in [a, b]$ are **nodes**, i.e. a finite number of points chosen in the integration interval,
- $w_k \in \mathbb{R}$ are **weights** (coefficients) chosen appropriately.

The choice of nodes and weights differentiates one numerical integration method from another, and different choices lead to different *degrees of precision*. We will see more about this next week.

### Riemann sums

You probably already know a numerical integration method -- the Riemann sum. Run the code cell below to display a figure (it uses [`matplotlib.patches.Rectangle()`](https://matplotlib.org/api/_as_gen/matplotlib.patches.Rectangle.html)):

*Remark:* The first command `%matplotlib notebook` is a notebook-wide setting, which allows to generate **dynamic plots** inside the Jupyter notebook, which we can e.g. zoom into or further modify. (We can toggle back to the default behaviour using the command `%matplotlib inline`, where all plots are "printed" for good when they are created, and cannot be further modified.)

Once you are finished with the plot you, you should click on the 'Stop interaction' blue button in the plot above.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

def f(x):
    return np.exp(-x**2)

# Create an x-axis with 100 points and estimate the function
a, b = 0, 3
x_plot = np.linspace(a, b, 100)
f_plot = f(x_plot)

# Create the nodes
N = 10
x_node = np.linspace(a, b, N+1)
f_node = f(x_node)

# Plot the function
fig, ax = plt.subplots(1, 2, figsize=(9, 3))

ax[0].plot(x_plot, f_plot, 'k-')
ax[1].plot(x_plot, f_plot, 'k-')

# Plot the rectangles for left and right sums
h = (b - a) / N
for k in range(N):
    rect = patches.Rectangle((x_node[k], 0), h, f_node[k], edgecolor='k')
    ax[0].add_patch(rect)
    
    rect = patches.Rectangle((x_node[k], 0), h, f_node[k+1], edgecolor='k')
    ax[1].add_patch(rect)
    
    
# Plot the nodes
ax[0].plot(x_node[:-1], f_node[:-1], 'rx')
ax[1].plot(x_node[1:], f_node[1:], 'rx')

# Label the plots
ax[0].set_xlabel(r'$x$')
ax[1].set_xlabel(r'$x$')
ax[0].set_ylabel(r'$f(x)$')

ax[0].set_title('Left Riemann sum')
ax[1].set_title('Right Riemann sum')

plt.show()

We can estimate the integral of $f(x)$ by calculating the area shaded in blue. Here, we subdivide the interval $[a, b]$ into $N$ partitions of equal width $h$:

$$
h = \frac{b-a}{N}
$$

The **nodes** are the end points of these sub-intervals, and here the **weights** are simply $h$, the width of each interval. The integral of $f(x)$ between $a$ and $b$ can then be estimated as:

$$
\begin{align}
\int_a^b f(x) \ dx &\approx \sum_{k=0}^{N-1} h \ f(x_k), \quad & \text{left Riemann sum} \\
\int_a^b f(x) \ dx &\approx \sum_{k=1}^N h \ f(x_k), \quad & \text{right Riemann sum}
\end{align}
$$

where the $N+1$ nodes $x_k$ are given by $x_k = a + kh$, with $k = 0, 1, \dots, N$. With this choice of nodes and weights, each element of the sum is simply the area of one blue rectangle.

In [ ]:
from math import erf
import numpy as np

# Estimate the integral
left_I = np.sum(h * f_node[:-1])
right_I = np.sum(h * f_node[1:])

# Exact value
I_exact = np.sqrt(np.pi) / 2 * (erf(b) - erf(a))

print(f'The exact integral is {I_exact:.3f}.\n')
print(f'The left Riemann sum is {left_I:.3f}.\n')
print(f'The right Riemann sum is {right_I:.3f}.\n')

---
🚩 ***Exercise 1:*** Using the Riemann sum methods above, estimate the value of the integral using different values of $N$. How does the accuracy change with $N$?

*Hint:* plot $\log(N)$ vs. $\log(\text{error})$. You may wish to use e.g. [`np.polyfit()`](https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html) or [`scipy.stats.linregress()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html).

In [ ]:
%run scripts/show_solutions.py W06-W1_ex1

---
### The midpoint rule

The midpoint rule is similar to the Riemann sums, but the nodes are taken as the **midpoint** of each partition instead of one of the extremities:

$$
\int_a^b f(x) \ dx \approx \sum_{k=0}^{N-1} h \ f(x_k),
$$

where the nodes $x_k$ are given by $x_k = a + \left(k + \frac{1}{2}\right)h$, with $k = 0, 1, \dots, N-1$.

In [ ]:
def f(x):
    return np.exp(-x**2)

# Create an x-axis with 100 points and estimate the function
a, b = 0, 3
x_plot = np.linspace(a, b, 100)
f_plot = f(x_plot)

# Create the nodes
N = 10
h = (b - a) / N
x_node = np.linspace(a + 0.5*h, b, N)
f_node = f(x_node)

# Plot the function
fig, ax = plt.subplots(figsize=(5, 3))
ax.plot(x_plot, f_plot, 'k-')

# Plot the rectangles
for k in range(N):
    rect = patches.Rectangle((x_node[k] - 0.5*h, 0), h, f_node[k], edgecolor='k')
    ax.add_patch(rect)

# Plot the nodes
ax.plot(x_node, f_node, 'rx')

# Label the plots
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f(x)$')
ax.set_title('Midpoint rule')

plt.show()

# Estimate the integral
midpoint_I = np.sum(h * f_node)

# Exact value
I_exact = np.sqrt(np.pi) / 2 * (erf(b) - erf(a))

print(f'The exact integral is {I_exact:.3f}.\n')
print(f'The estimated integral using the midpoint rule is {midpoint_I:.3f}.\n')

---
🚩 ***Exercise 2:*** Using the midpoint rule method above, estimate the value of the integral using different values of $N$. How does the accuracy change with $N$?

In [ ]:
%run scripts/show_solutions.py W06-W1_ex2

---
### The trapezoid rule

The trapezoid rule also uses partitions of equal width, but instead of approximating the integral as the area of rectangles, it uses trapezoids -- the function is interpolated linearly between the nodes.

$$
\int_a^b f(x) \ dx \approx \sum_{k=0}^{N-1} h\frac{\left(f(x_k) + f(x_{k+1})\right)}{2} ,
$$

where the nodes $x_k$ are given by $x_k = a +kh$, with $k = 0, 1, \dots, N$.

In [ ]:
def f(x):
    return np.exp(-x**2)

# Create an x-axis with 100 points and estimate the function
a, b = 0, 3
x_plot = np.linspace(a, b, 100)
f_plot = f(x_plot)

# Create the nodes
N = 6
h = (b - a) / N
x_node = np.linspace(a, b, N + 1)
f_node = f(x_node)

# Plot the function
fig, ax = plt.subplots(figsize=(5, 3))
ax.plot(x_plot, f_plot, 'k-')

# Plot the trapezoids
for k in range(N):
    verts = [[x_node[k], 0], [x_node[k+1], 0],
             [x_node[k+1], f_node[k+1]], [x_node[k], f_node[k]]]
    trapz = patches.Polygon(verts, h, edgecolor='k')
    ax.add_patch(trapz)

# Plot the nodes
ax.plot(x_node, f_node, 'rx')

# Label the plots
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f(x)$')
ax.set_title('Trapezoid rule')

plt.show()

# Estimate the integral
midpoint_I = np.sum(0.5 * h * (f_node[:-1] + f_node[1:]))

# Exact value
I_exact = np.sqrt(np.pi) / 2 * (erf(b) - erf(a))

print(f'The exact integral is {I_exact:.3f}.\n')
print(f'The estimated integral using the midpoint rule is {midpoint_I:.3f}.\n')

---
🚩 ***Exercise 3:*** Using the trapezoid rule method above, estimate the value of the integral using different values of $N$. How does the accuracy change with $N$?

In [ ]:
%run scripts/show_solutions.py W06-W1_ex3